In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import os,csv,re,time
import requests
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
from datetime import datetime

In [ ]:
def save_full_screenshot(driver, regnumber=""):
    try:
        os.makedirs("screenshots", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"screenshots/{regnumber}_{timestamp}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")

# def save_screeshort(driver,reg=""):
#     try:
#         os.makedirs("screensort",exist_ok=True)
#         timestamp = datetime.now().strftime("%y%m%d_%H%M%S")
#         filename = f"screenshots/{reg}_{timestamp}.png"
#         driver.save_screensort(filename)
#         print(f"Screensort saved: {filename} ")
#     except Exception as e:
#         print(f"X screen short faild {e}")

In [ ]:
def scrape(path):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 20)
    driver.get(path)

    results = []
    try:
        cookie_accept = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept') or contains(@class, 'accept')]"))
        )
        driver.execute_script("arguments[0].click();", cookie_accept)
        print("✅ Cookies accepted successfully")
    except TimeoutException:
        print("⚠️ No cookie popup found or already accepted")
    try:
        menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.nav-toggle.js-subnav-toggle")))
        driver.execute_script("arguments[0].click();", menu)
        print("✅ User dropdown opened")

        login_link = wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/login')]")))
        driver.execute_script("arguments[0].scrollIntoView(true);", login_link)
        driver.execute_script("arguments[0].click();", login_link)
        print("✅ Clicked login link")
    except Exception as e:
        print(f"⚠️ Could not open login page: {e}")
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        provided_pass = "Muhssan7865"

        user_name = wait.until(EC.presence_of_element_located((By.ID, "Email")))
        user_name.clear()
        user_name.send_keys(provided_u_name)

        password = wait.until(EC.presence_of_element_located((By.ID, "Password")))
        password.clear()
        password.send_keys(provided_pass)

        try:
            terms = driver.find_element(By.ID, "IsAgreeToTerms")
            driver.execute_script("arguments[0].click();", terms)
        except:
            pass

        submit_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit' and contains(., 'Login')]")))
        driver.execute_script("arguments[0].click();", submit_btn)
        print("✅ Login submitted")
    except Exception as e:
        print(f"❌ Login error: {e}")
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".user-nav .js-subnav-toggle")))
        print("🎉 Login successful — dashboard loaded.")
    except TimeoutException:
        print("⚠️ Login may not have completed — continuing anyway.")
    try:
        num_cars_text = wait.until(EC.presence_of_element_located(
            (By.XPATH, '(.//p[@class="pager__label"])[1]//span'))).text
        cars = int(num_cars_text.split(" ")[-1])
        print(f"📦 Total cars found: {cars}")
    except:
        cars = 0
        print("⚠️ Couldn't detect number of cars — will loop until no more cars.")
    try:
        first_link = wait.until(EC.element_to_be_clickable(
            (By.XPATH, '(//a[contains(@href,"/lot/")])[1]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", first_link)
        driver.execute_script("arguments[0].click();", first_link)
        print("✅ Opened first car")
    except Exception as e:
        print(f"❌ Could not click first car: {e}")
    car_index = 0
    while True:
        car_index += 1
        print(f"\n🚗 Scraping car {car_index}")

        details = {}

        # Title
        try:
            details["Title"] = wait.until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="lot-title-header__item"]//h1'))
            ).text.strip()
        except:
            details["Title"] = "N/A"

        # Lot
        try:
            lot_text = driver.find_element(By.XPATH, '//span[contains(@class,"lotno")]').text.strip()
            details["Lot"] = lot_text.split(" ")[-1]
        except:
            details["Lot"] = "N/A"

        # Auction End
        try:
            auc_end = driver.find_element(By.XPATH, '//p[@class="lot-title-header__ends"]').text.strip()
            if ": " in auc_end:
                details["Auction Ends"] = auc_end.split(": ")[1]
        except:
            details["Auction Ends"] = "N/A"

        # Car Info (Specs)
        try:
            info_rows = driver.find_elements(By.XPATH, '//li[contains(@class,"listing-properties__list-item")]')
            for row in info_rows:
                key = row.find_element(By.XPATH, './/p[@class="listing-properties__name"]').text.strip()
                val = row.find_element(By.XPATH, './/p[@class="listing-properties__value"]').text.strip()
                details[key] = val
            save_full_screenshot(driver,details["Registration Number"])
        except:
            pass

        # Images
        try:
            
            driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(1)
            gallery_btn = driver.find_element(By.XPATH, '//button[contains(@class,"btn--lot-gallery")]')
            driver.execute_script("arguments[0].click();", gallery_btn)

            imgs = wait.until(EC.presence_of_all_elements_located(
                (By.XPATH, '//img[contains(@class,"gallery__modal-gallery-image")]')))
            img_urls = [img.get_attribute("src") for img in imgs if img.get_attribute("src")]
            details["Images"] = ", ".join(img_urls)

            close_btn = driver.find_element(By.XPATH, '//button[@id="gallery-modal-close"]')
            driver.execute_script("arguments[0].click();", close_btn)
        except:
            details["Images"] = "N/A"

        # Save one car
        results.append(details)
        print(f"✅ Car scraped: {details['Title']}")

        # Try next
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[contains(@class,"btn-next-lot")]')))
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
        except:
            print("✅ No more cars found.")
            break

    # ==== Save CSV ====
    df = pd.DataFrame(results)
    df.to_csv("brightwells.csv", index=False, encoding="utf-8-sig")
    print("\n🎉 All cars scraped successfully! Saved to brightwells_results.csv")

    driver.quit()

# Run
path = "https://www.brightwells.com/timed-sale/5781"
scrape(path)

✅ Cookies accepted successfully
✅ User dropdown opened
✅ Clicked login link
✅ Login submitted
🎉 Login successful — dashboard loaded.
⚠️ Couldn't detect number of cars — will loop until no more cars.
✅ Opened first car

🚗 Scraping car 1
📸 Screenshot saved: screenshots/BRB354_20251110_182906.png
✅ Car scraped: 1935 Ford Model AA 30 CWT

🚗 Scraping car 2
📸 Screenshot saved: screenshots/OPH596E_20251110_182912.png
✅ Car scraped: 1967 Mercedes-Benz 250SE (W108)

🚗 Scraping car 3
📸 Screenshot saved: screenshots/475CXU_20251110_182917.png
✅ Car scraped: 1961 Lancia Appia

🚗 Scraping car 4
📸 Screenshot saved: screenshots/NV9240_20251110_182923.png
✅ Car scraped: 1937 Austin Seven Ruby

🚗 Scraping car 5
📸 Screenshot saved: screenshots/A455GLW_20251110_182928.png
✅ Car scraped: 1983 Ford Capri 2.8i

🚗 Scraping car 6
📸 Screenshot saved: screenshots/RMD760L_20251110_182933.png
✅ Car scraped: 1973 BMW 2002

🚗 Scraping car 7
📸 Screenshot saved: screenshots/HMF386K_20251110_182938.png
✅ Car scraped: 

In [4]:
df=pd.read_csv("brightwells.csv")
df.columns

Index(['Title', 'Lot', 'Auction Ends', 'Registration Number', 'Engine Size',
       'Engine Number', 'Chassis Number', 'Gearbox', 'VAT Status',
       'Buyers Premium (excl. VAT)', 'Documents', 'Tax and MOT status',
       'Images'],
      dtype='object')

In [5]:
def remove_trailing_na_rows_inplace(input_csv, na_values=None, cutoff_by_first_col=False):
    """
    Cleans a CSV file *in place* — removes all rows starting from the first
    where either:
      - all columns are blank/N/A (default), OR
      - the first column is blank/N/A (if cutoff_by_first_col=True)
    """

    if na_values is None:
        na_values = {"n/a", "na", ""}

    # Load CSV as strings (don’t auto-convert NA)
    try:
        df = pd.read_csv(input_csv, dtype=str, keep_default_na=False)
    except Exception:
        df = pd.read_csv(input_csv, dtype=str, keep_default_na=False, engine="python")

    # Normalize text for checking
    norm = df.fillna("").astype(str).applymap(lambda x: x.strip().lower())

    # Helper to test NA
    def is_na_cell(val):
        return val in na_values

    cutoff_idx = None
    if cutoff_by_first_col:
        # Cut when first column becomes N/A
        first_col = norm.columns[0]
        for idx, cell in enumerate(norm[first_col].tolist()):
            if is_na_cell(cell):
                cutoff_idx = idx
                break
    else:
        # Cut when *all* columns are N/A
        for idx, row in enumerate(norm.values):
            if all(is_na_cell(cell) for cell in row):
                cutoff_idx = idx
                break

    if cutoff_idx is not None:
        print(f"🧹 Removing rows from index {cutoff_idx} onwards ({len(df) - cutoff_idx} rows deleted).")
        df = df.iloc[:cutoff_idx]
    else:
        print("✅ No trailing N/A rows found — nothing removed.")

    # Save back to same file (in-place)
    df.to_csv(input_csv, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"💾 Cleaned and overwritten: {input_csv}")
remove_trailing_na_rows_inplace("brightwells.csv")

🧹 Removing rows from index 7 onwards (1 rows deleted).
💾 Cleaned and overwritten: brightwells.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_7128\1048111810.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  norm = df.fillna("").astype(str).applymap(lambda x: x.strip().lower())


### image downloader

In [ ]:
from urllib.parse import urlparse, urljoin
from PIL import Image, ImageDraw, ImageFont


def add_watermark_to_image(image_path, text="Sourced from Brightwells"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)
        font_size = max(20, image.width // 50)  
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()
        margin = int(font_size * 0.6)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin
        box_padding = int(font_size * 0.4)
        draw.rectangle(
            [x - box_padding, y - box_padding, x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 180)
        )
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 240))
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")

    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")


def download_images(data, main_folder="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = str(row["Registration Number"]).strip()
        image_urls = row["Images"]
        
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)

        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=10)
                response.raise_for_status()

                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1].lower()

                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"

                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")

                with open(full_file_name, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                print(f"📸 Downloaded: {full_file_name}")
                add_watermark_to_image(full_file_name)

            except Exception as e:
                print(f"❌ Failed to download {url} for {reg_no}: {e}")


# Call the function
df = pd.read_csv("brightwells.csv")
reg_img = df[["Registration Number", "Images"]]
download_images(reg_img)

📸 Downloaded: Images\BRB354\BRB354_1_image_1.jpg
✅ Watermark added to Images\BRB354\BRB354_1_image_1.jpg
📸 Downloaded: Images\BRB354\BRB354_2_image_2.jpg
✅ Watermark added to Images\BRB354\BRB354_2_image_2.jpg
📸 Downloaded: Images\BRB354\BRB354_3_image_3.jpg
✅ Watermark added to Images\BRB354\BRB354_3_image_3.jpg
📸 Downloaded: Images\BRB354\BRB354_4_image_4.jpg
✅ Watermark added to Images\BRB354\BRB354_4_image_4.jpg
📸 Downloaded: Images\BRB354\BRB354_5_image_5.jpg
✅ Watermark added to Images\BRB354\BRB354_5_image_5.jpg
📸 Downloaded: Images\BRB354\BRB354_6_image_6.jpg
✅ Watermark added to Images\BRB354\BRB354_6_image_6.jpg
📸 Downloaded: Images\BRB354\BRB354_7_image_7.jpg
✅ Watermark added to Images\BRB354\BRB354_7_image_7.jpg
📸 Downloaded: Images\BRB354\BRB354_8_image_8.jpg
✅ Watermark added to Images\BRB354\BRB354_8_image_8.jpg
📸 Downloaded: Images\BRB354\BRB354_9_image_9.jpg
✅ Watermark added to Images\BRB354\BRB354_9_image_9.jpg
📸 Downloaded: Images\BRB354\BRB354_10_image_10.jpg
✅ Wa